<a href="https://colab.research.google.com/github/leejunho12316/HonGongMachine/blob/main/%EB%B9%85%EB%B6%84%EA%B8%B0_%EC%8B%A4%EA%B8%B0_Part4_%EC%A0%9C2%ED%9A%8C_%EA%B8%B0%EC%B6%9C%EB%AC%B8%EC%A0%9Cipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#제1유형

In [1]:
import pandas as pd
df = pd.read_csv('members.csv')

In [13]:
#문제1
df = df.sort_values('views',ascending=False)

s = df.iloc[9,9]
df.iloc[0:10, 9] = s

res = round(df[df['age'] >= 80]['views'].mean(), 2)
print(res)

5674.04


In [41]:
#문제2
import pandas as pd
df=pd.read_csv('members.csv')

df = df.iloc[:int(len(df)*0.8), : ]

std_before = df['f1'].std()

df['f1'] = df['f1'].fillna(df['f1'].median())
std_after = df['f1'].std()

print(round(abs(std_before - std_after), 2))

3.56


In [45]:
#문제3
import pandas as pd
df=pd.read_csv('members.csv')

std = df['age'].std()
mean = df['age'].mean()

cond1 = df['age'] < mean-std*1.5
cond2 = mean+std*1.5 < df['age']

df[cond1 | cond2]['age'].sum()

np.float64(473.5)

#제2유형

In [186]:
import pandas as pd
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_test = pd.read_csv('X_test.csv')

In [187]:
print(X_train.shape, X_test.shape, y_train.shape)
print(X_train.isnull().sum().sum(), X_test.isnull().sum().sum(), y_train.isnull().sum().sum())

(8799, 11) (2200, 11) (8799, 2)
0 0 0


데이터 전처리

In [188]:
#ID 삭제
X_train.drop('ID', axis=1, inplace=True)
X_test_id = X_test.pop('ID')

data = pd.concat([X_train, X_test], axis=0)

#OH Encoding
#data = pd.get_dummies(data)

#Label Encoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

data['Warehouse_block'] = le.fit_transform(data['Warehouse_block'])
data['Mode_of_Shipment'] = le.fit_transform(data['Mode_of_Shipment'])
data['Product_importance'] = le.fit_transform(data['Product_importance'])
data['Gender'] = le.fit_transform(data['Gender'])

#Cost of the Product 스케일링
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
data[['Cost_of_the_Product']] = scaler.fit_transform(data[['Cost_of_the_Product']])
data[['Weight_in_gms']] = scaler.fit_transform(data[['Weight_in_gms']])

X_train = data.iloc[:len(X_train), ]
X_test = data.iloc[len(X_train):, ]

In [189]:
y_train = y_train['Reached.on.Time_Y.N']

In [190]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state = 99)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(6159, 10) (2640, 10) (6159,) (2640,)


모델학습

In [201]:
#1. 로지스틱 회귀
from sklearn.linear_model import LogisticRegression
model1 = LogisticRegression(random_state = 99)
model1.fit(X_train, y_train)

from sklearn.metrics import roc_auc_score
pred = model1.predict_proba(X_val)
score = roc_auc_score(y_val, pred[:,1])
print('1. 로지스틱 회귀', score)

#2. 랜덤 포레스트
from sklearn.ensemble import RandomForestClassifier
model2 = RandomForestClassifier(random_state = 99)
model2.fit(X_train, y_train)

from sklearn.metrics import roc_auc_score
pred = model2.predict_proba(X_val)
score = roc_auc_score(y_val, pred[:,1])
print('2. 랜덤 포레스트', score)

#3. 의사결정 나무
from sklearn.tree import DecisionTreeClassifier
model3 = DecisionTreeClassifier(random_state = 99)
model3.fit(X_train, y_train)

from sklearn.metrics import roc_auc_score
pred = model3.predict_proba(X_val)
score = roc_auc_score(y_val, pred[:,1])
print('3. 의사결정 나무', score)

#4. XGBoost
import xgboost as xgb
model4 = xgb.XGBClassifier(random_state = 99)
model4.fit(X_train, y_train)

from sklearn.metrics import roc_auc_score
pred = model4.predict_proba(X_val)
score = roc_auc_score(y_val, pred[:,1])
print('4. XGBoost : ', score)

#5. lightGBM
import lightgbm as lgb
model5 = lgb.LGBMClassifier(random_state = 99)
model5.fit(X_train, y_train)

from sklearn.metrics import roc_auc_score
pred = model5.predict_proba(X_val)
score = roc_auc_score(y_val, pred[:,1])
print('5. Light GBM : ', score)



1. 로지스틱 회귀 0.7314995315291501
2. 랜덤 포레스트 0.7500064901503118
3. 의사결정 나무 0.6349662158175584
4. XGBoost :  0.7461212501681539
[LightGBM] [Info] Number of positive: 3698, number of negative: 2461
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000273 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 566
[LightGBM] [Info] Number of data points in the train set: 6159, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.600422 -> initscore=0.407224
[LightGBM] [Info] Start training from score 0.407224
5. Light GBM :  0.747848220164779


In [ ]:
#1. id삭제, oh, minmax : 0.731069411567572
#2. id삭제, oh, robust : 0.7313561582086241
#3. id삭제, le, minmax : 0.7312729662818991
----
#4. id삭제, le, robust : 0.7314995315291501
# -> 랜덤 포레스트 0.7500064901503118

#저장

In [202]:
res = model2.predict_proba(X_test)
res_df = pd.DataFrame({
    'ID' : X_test_id,
    'Reached.on.Time_Y.N' : res[:,1]
    })
res_df.to_csv('result.csv', index=False)

In [203]:
load = pd.read_csv('result.csv')
load

,ID,Reached.on.Time_Y.N
0,8285,0.41
1,10192,0.61
2,8675,0.37
3,5753,0.54
4,448,0.96
...,...,...
2195,7193,0.42
2196,4445,0.37
2197,7390,0.49
2198,9977,0.25
